In [1]:
pre_trained_model = 'resnet101'

### 라이브러리 불러오기

In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from torch.utils.data.dataset import random_split

import gc
gc.collect()
torch.cuda.empty_cache()

from PIL import Image

device = torch.device('cuda:0')

In [3]:
pwd()

'/home/ubuntu/AD_Heewon'

In [25]:
## 작업경로 확인
#path = '/home/nextgen/Downloads/Heewon_download/data'

#os.chdir(path)
#os.getcwd()

In [13]:
train_png = sorted(glob('open/train/*.png'))
test_png = sorted(glob('open/test/*.png'))

In [15]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [16]:
# 호영's code

index_good = train_y[train_y['state'] == 'good'].index
index_bad = [i for i in list(np.arange(len(train_labels))) if i not in list(index_good)]

for i in index_bad:
    train_labels.append(train_labels[i]) #품질 나쁜 리스트 추가
    train_labels.append(train_labels[i]) 
    train_labels.append(train_labels[i]) 
    train_labels.append(train_labels[i]) 
    train_labels.append(train_labels[i]) 

In [17]:
def img_load(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (512, 512))
    return img

In [18]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:20<00:00, 26.81it/s]


In [19]:
#호영's code
height, width = train_imgs[0].shape[:2]

img90 = cv2.getRotationMatrix2D((height/2, width/2), 90 ,1)
img180 = cv2.getRotationMatrix2D((height/2, width/2), 180 ,1)
img270 = cv2.getRotationMatrix2D((height/2, width/2), 270 ,1)

train_png_bad = []
for i in index_bad:
    train_png_bad.append(train_png[i]) #품질 나쁜 리스트의 사진을 추가
    
train_imgs_bad = []
for j in [90,180,270]:
    for i in tqdm(train_png_bad):
        img = img_load(i)
        train_imgs_bad.append(cv2.warpAffine(img, globals()['img'+str(j)], (width, height)))

train_imgs.extend(train_imgs_bad)

100%|██████████| 648/648 [00:26<00:00, 24.33it/s]


In [ ]:
#호영's code

height, width = train_imgs[0].shape[:2]

pts1 = np.float32([[0,0],[width-1,0],[0,height-1]])
pts2 = np.float32([[0,0],[width-25,25],[25, height-25]])

train_png_bad = []
for i in index_bad:
    train_png_bad.append(train_png[i]) #품질 나쁜 리스트의 사진을 추가
    
train_imgs_bad = []
for i in tqdm(train_png_bad):
    mat = cv2.getAffineTransform(pts1,pts2)
    img = img_load(i)
    train_imgs_bad.append(cv2.warpAffine(img, mat, None)) #행렬 변환을 통한 그림 생성

train_imgs.extend(train_imgs_bad)

In [ ]:
#호영's code

height, width = train_imgs[0].shape[:2]

pts1 = np.float32([[0,0],[width-1,0],[0,height-1]])
pts2 = np.float32([[0,0],[width+25,-25],[-25, height+25]])

train_png_bad = []
for i in index_bad:
    train_png_bad.append(train_png[i]) #품질 나쁜 리스트의 사진을 추가
    
train_imgs_bad = []
for i in tqdm(train_png_bad):
    mat = cv2.getAffineTransform(pts1,pts2)
    img = img_load(i)
    train_imgs_bad.append(cv2.warpAffine(img, mat, None)) #행렬 변환을 통한 그림 생성

train_imgs.extend(train_imgs_bad)

In [21]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model(pre_trained_model, pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [42]:
batch_size = 16
epochs = 50

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

### 모델 학습

In [43]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 
# best=0

In [44]:
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

epoch : 1/50    time : 78s/3826s
TRAIN    loss : 1.71524    f1 : 0.14959
epoch : 2/50    time : 80s/3818s
TRAIN    loss : 0.90131    f1 : 0.27237
epoch : 3/50    time : 79s/3707s
TRAIN    loss : 0.75988    f1 : 0.34178
epoch : 4/50    time : 79s/3642s
TRAIN    loss : 0.64236    f1 : 0.44179
epoch : 5/50    time : 80s/3594s
TRAIN    loss : 0.53525    f1 : 0.53620
epoch : 6/50    time : 78s/3420s
TRAIN    loss : 0.47765    f1 : 0.59587
epoch : 7/50    time : 79s/3411s
TRAIN    loss : 0.42331    f1 : 0.62237
epoch : 8/50    time : 79s/3314s
TRAIN    loss : 0.37132    f1 : 0.68796
epoch : 9/50    time : 79s/3223s
TRAIN    loss : 0.35822    f1 : 0.70603
epoch : 10/50    time : 78s/3121s
TRAIN    loss : 0.29136    f1 : 0.74105
epoch : 11/50    time : 79s/3068s
TRAIN    loss : 0.23728    f1 : 0.79238
epoch : 12/50    time : 80s/3042s
TRAIN    loss : 0.22738    f1 : 0.81514
epoch : 13/50    time : 80s/2959s
TRAIN    loss : 0.22241    f1 : 0.82077
epoch : 14/50    time : 79s/2854s
TRAIN    loss

### 추론

In [45]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [46]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [47]:
submission = pd.read_csv("open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-misplaced
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-good
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [54]:
pwd()

'/home/ubuntu/AD_Heewon'

In [51]:
output_file = "open/result/{}(batch={}, epoch={}).csv".format(pre_trained_model, batch_size, epochs)

In [55]:
submission.to_csv("open/resnet101(batch=16, epoch=50).csv", index = False)